In [1]:
import re as re
import time
import zipcode
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

import time
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
#zipcode.islike: list of Zip objects that begin with given prefix.
def find_all_zipcode(zip_items):
    #single sting
    if type(zip_items) == str:
        zipcode_obj = zipcode.islike(zip_items)
        output = re.findall('\d+', str(zipcode_obj))
    elif type(zip_items) == list:
        output = [n for each in zip_items for n in re.findall('\d+', str(zipcode.islike(each)))]       
    else:
        raise ValueError("input 'zip_items' must be of type str or list")
    return output



In [3]:
def open_driver(driverpath):
    driver = webdriver.Chrome(driverpath)
    driver.get('https://www.zillow.com/homes/for_rent/') 
    return driver

def input_grab(driver,search_term):
    wait = WebDriverWait(driver, 10)
    
    actions = wait.until(EC.presence_of_element_located((By.ID, "citystatezip")))
    button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "zsg-icon-searchglass")))
    actions.clear()
    time.sleep(3)
    actions.send_keys(search_term)
    time.sleep(3)
    button.click()
    time.sleep(3)  
    
    output = []
    while True:
    # grab the data
        output.append(driver.page_source)
        # click next link
        try:             
            element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'zsg-pagination-next')))
            element.click()
            time.sleep(3)
        except TimeoutException:
            break
    return output



In [4]:
def get_listings(list_obj):
    # Split the raw HTML into segments, one for each listing.
    output = []
    for i in list_obj:
        htmlSplit = i.split('" id="zpid_')[1:]
        output += htmlSplit
    print 
    print str(len(output)) , " results scraped\n***"
    return output


In [8]:
find_all_zipcode('0703')

['07030', '07031', '07032', '07033', '07034', '07035', '07036', '07039']

In [5]:
driver = open_driver('/Users/Lucinda/Downloads/chromedriver')

In [10]:
raw_data = input_grab(driver, '07030')

In [11]:
listings = get_listings(raw_data)


476  results scraped
***


In [12]:
Street_Address = []
City = []
State = []
Zip_Code = []
Price = []
SQFT = []
Bedrooms = []
Bathrooms = []
Days_on_Market = []
URL = []

for i in  range(len(listings)): 
    soup = BeautifulSoup(listings[i], "lxml")    
    try:
        Street_Address.append(soup.find('span', {"itemprop" : "streetAddress"}).get_text())
    except AttributeError:
        Street_Address.append("NA")            
    try:
        City.append(soup.find('span', {"itemprop": "addressLocality"}).get_text())
    except AttributeError:
        City.append("NA")        
    try:
        State.append(soup.find('span', {"itemprop": "addressRegion"}).get_text())
    except AttributeError:
        State.append("NA")     
    try:
        Zip_Code.append(soup.find('span', {"itemprop": "postalCode"}).get_text())
    except AttributeError:
        Zip_Code.append("NA")
    try:
        Price.append(
            int(soup.find('span', {"class": "zsg-photo-card-price"}).get_text().split('/')[0].replace('$', '').replace(',', '')))
    except AttributeError:
        Price.append("NA")
    
    try: 
        #info = soup.find('span', {"class": "zsg-photo-card-info"}).get_text()
        Bedrooms.append([int(s) for s in soup.find('span', {"class": "zsg-photo-card-info"}).get_text().replace(',', '').split() if s.isdigit()][0])
    except (ValueError, IndexError,AttributeError):
        Bedrooms.append("NA")
    
    try: 
        Bathrooms.append([int(s) for s in soup.find('span', {"class": "zsg-photo-card-info"}).get_text().replace(',', '').split() if s.isdigit()][1])

    except (ValueError, IndexError,AttributeError):
        Bathrooms.append("NA")
    
    try:   
        SQFT.append([int(s) for s in soup.find('span', {"class": "zsg-photo-card-info"}).get_text().replace(',', '').split() if s.isdigit()][2])
    except (ValueError, IndexError,AttributeError):
        SQFT.append("NA")
        
    try:   
        Days_on_Market.append(soup.find('span', {"class": "zsg-photo-card-notification"}).get_text())
    except (ValueError, IndexError,AttributeError):
        Days_on_Market.append("NA")
    

columns={'streetAddress':Street_Address, 'City': City, 'State':State, 'Zip_Code': Zip_Code, 'Monlthly Rental': Price,
        'SQFT' : SQFT, 'Bedrooms' : Bedrooms, 'Bathrooms' : Bathrooms, ' Days_on_Market':  Days_on_Market}

#print len(Street_Address),len(City), len(State), len(Zip_Code)
# Create a dataframe from the columns variable
df = pd.DataFrame(columns)

In [13]:
df.head()

,Days_on_Market,Bathrooms,Bedrooms,City,Monlthly Rental,SQFT,State,Zip_Code,streetAddress
0,6 minutes ago,2,4,Hoboken,3750,1150,NJ,07030,11th St
1,8 minutes ago,1,2,Hoboken,3100,985,NJ,07030,561 1st St # 2
2,Updated today,NA,NA,NA,NA,NA,NA,NA,NA
3,Updated today,NA,NA,NA,NA,NA,NA,NA,NA
4,Updated today,NA,NA,NA,NA,NA,NA,NA,NA
5,2 hours ago,2,3,Hoboken,3850,1300,NJ,07030,520 Park Ave APT 4
6,2 hours ago,1,1,Hoboken,2350,NA,NJ,07030,Garden St
7,3 hours ago,1,1,Hoboken,2250,750,NJ,07030,Washington St
8,3 hours ago,2,2,Hoboken,2850,1000,NJ,07030,402 Jefferson St # 510
9,4 hours ago,1,1,Hoboken,2450,775,NJ,07030,Garden St
